In [ ]:
!pip install tensorflow

In [ ]:
import notebook
notebook.__version__

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data_directory = "C:\\Users\\USER\\Desktop\\dataset\\Train" #You have to change this depends on where you place the dataset

# Create the data augmentation generator for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2 # Set the validation split
)

# Prepare the data generators
train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training' # Set as training data
)

validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation' # Set as validation data
)

# Get the number of classes
num_classes = len(train_generator.class_indices)

In [ ]:
# Load the pre-trained model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create a new model using the base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Freeze the weights of the base model during training
for layer in base_model.layers:
    layer.trainable = False

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
epochs = 10 

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

epochs = 10

# Train the model using data generators
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=epochs,
    callbacks=[early_stopping]
)

In [ ]:
loss, accuracy = model.evaluate(validation_generator)

In [ ]:
# Convert the custom model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Define the desired file path for saving the TensorFlow Lite model
file_path = "C:\\Users\\USER\\Desktop\\dataset\\mobilenet.tflite"

# Save the TensorFlow Lite model to the specified path
with open(file_path, "wb") as f:
    f.write(tflite_model)

In [ ]:
class_labels = os.listdir(data_directory)
labels_txt_path = "C:\\Users\\USER\\Desktop\\dataset\\labels.txt"

with open(labels_txt_path, "w") as f:
    for label in class_labels:
        f.write(f"{label}\n")

## Testing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load the TFLite model
tflite_model_file = "C:\\Users\\USER\\Desktop\\dataset\\mobilenet.tflite"

with open(tflite_model_file, 'rb') as f:
    tflite_model = f.read()

In [ ]:
# Load the TFLite model using the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import efficientnet.tfkeras as efn

count = 0
count_thresh = 0
count_total = 0
threshold = 0.9
dir_path = "C:\\Users\\USER\\Desktop\\dataset\wrong"

# Calculate the number of rows needed for the plot
n_images = len(os.listdir(dir_path))
n_columns = 5
n_rows = n_images // n_columns + (n_images % n_columns > 0)

# Create the figure and set its size
fig = plt.figure(figsize=(20, 4 * n_rows))

for idx, i in enumerate(os.listdir(dir_path), start=1):
    img = image.load_img(dir_path + '//' + i, target_size=(224, 224))

    # Display the image in a subplot
    plt.subplot(n_rows, n_columns, idx)
    plt.imshow(img)
    plt.axis('off')

    count_total += 1

    # Preprocess the image
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = efn.preprocess_input(X)

    # Make a prediction
    interpreter.set_tensor(input_details[0]['index'], X)
    interpreter.invoke()
    val = interpreter.get_tensor(output_details[0]['index'])
    prediction = np.argmax(val)  # Get the index of the maximum value in the val array
    max_prob = np.max(val)  # Get the maximum probability in the val array

    if max_prob < threshold:
        title = "no target attraction"
        count_thresh += 1

    else:
        if prediction == 0:
            title = "Block Sign"

        elif prediction == 1:
            title = "Ceiling Artifacts"
            
        elif prediction == 2:
            title = "Corridor"
        elif prediction == 3:
            title = "Ink Block"

        elif prediction == 4:
            title = "Inviting Reception Pavilion"

        elif prediction == 5:
            title = "Main Entrance"

        elif prediction == 6:
            title = "Moon Gate"

        else:
            title = "The Gesture"
            count += 1
            

    # Set the title of the subplot
    plt.title(title)

# Show the complete figure
plt.tight_layout()
plt.show()

print("Count for correct:",count)
print("Count for below threshold:",count_thresh)
print("Total:",count_total)